In [18]:
from main import Solver
import torch
import numpy as np
import pandas as pd

# Hyper Params Section
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using", torch.cuda.device_count(), "GPUs")


Exp_label = pd.read_csv('RealData/Exp664_genes.txt', sep='\t',header = None)
Exp_list = Exp_label.iloc[:, 0].values.tolist()
Exp = pd.DataFrame(np.loadtxt("RealData/Exp664.txt").T,columns = Exp_label)

Meth_label = pd.read_csv('RealData/Meth664_probes.txt', sep='\t',header = None)
Meth_list = Meth_label.iloc[:, 0].values.tolist()
Meth = pd.DataFrame(np.loadtxt("RealData/Meth664.txt").T,columns = Meth_label)

miRNA_label = pd.read_csv('RealData/miRNA664_miRNA.txt', sep='\t',header = None)
miRNA_list = miRNA_label.iloc[:, 0].values.tolist()
miRNA = pd.DataFrame(np.loadtxt("RealData/miRNA664.txt").T,columns = miRNA_label)

y = pd.read_csv('RealData/PAM50label664.txt',header = None)

Exp_value = np.loadtxt("RealData/Exp664.txt")
Meth_value = np.loadtxt("RealData/Meth664.txt")
miRNA_value = np.loadtxt("RealData/miRNA664.txt")
views = [torch.tensor(Exp_value).T,torch.tensor(Meth_value).T,torch.tensor(miRNA_value).T]

print(f'input views shape :')
for i, view in enumerate(views):
    print(f'view_{i} :  {view.shape}')
    view = view.to("cpu")

Solver = Solver(device)

Using 0 GPUs
input views shape :
view_0 :  torch.Size([664, 2642])
view_1 :  torch.Size([664, 3298])
view_2 :  torch.Size([664, 437])


In [19]:
## train hyper

b0, obj = Solver.tune_hyper(x_list=train_data, set_params=3,iter=50)
print(b0)

for rep in range(5):
    print("REP=", rep + 1)
    
    ## fit results
    u = Solver._get_outputs(train_data, 1e-7, 100, b0)
    u_list.append(u)

Start Hyperparams Tuning
Sparsity selection number= 1 hyperparams= (1.0, 1.0, 1.0) obj= tensor(0.0601)
Sparsity selection number= 2 hyperparams= (1.0, 1.0, 7.071067811865475) obj= tensor(0.0647)
Sparsity selection number= 3 hyperparams= (1.0, 1.0, 49.99999999999999) obj= tensor(0.0880)
Sparsity selection number= 4 hyperparams= (1.0, 7.071067811865475, 7.071067811865475) obj= tensor(0.1709)
Sparsity selection number= 5 hyperparams= (1.0, 7.071067811865475, 49.99999999999999) obj= tensor(0.1730)
Sparsity selection number= 6 hyperparams= (1.0, 49.99999999999999, 49.99999999999999) obj= tensor(0.1873)
Sparsity selection number= 7 hyperparams= (7.071067811865475, 7.071067811865475, 7.071067811865475) obj= tensor(0.2034)
Sparsity selection number= 8 hyperparams= (7.071067811865475, 7.071067811865475, 49.99999999999999) obj= tensor(0.2063)
Sparsity selection number= 9 hyperparams= (7.071067811865475, 49.99999999999999, 49.99999999999999) obj= tensor(0.2216)
Sparsity selection number= 10 hyper

In [20]:
print(u_list[0][0])
## obj = 0.1489
## post processing
u_best = u_list[3]

tensor([[ 0.0160],
        [-0.0581],
        [ 0.0025],
        ...,
        [ 0.0000],
        [ 0.0175],
        [-0.0000]])


In [55]:
import os
## post processing

Exp_total = []
Meth_total = []
miRNA_total = []
for i in range(5):
    u_best = u_list[i]

    Exp_label = pd.read_csv('RealData/Exp664_genes.txt', sep='\t',header = None)
    rows, _ = np.where(np.array(u_best[0]) > 0.05)
    Labels = Exp_label.iloc[rows,:].reset_index(drop=True)
    Scores = pd.DataFrame(u_best[0][rows]).reset_index(drop=True)
    Exp_table = pd.concat([Labels, Scores],axis=1)
    Exp_table.columns = ['genes','scores']
    Exp_table = Exp_table.sort_values(by='scores', key=abs, ascending=False)

    Exp_total.append(Exp_table)

    Meth_label = pd.read_csv('RealData/Meth664_probes.txt', sep='\t',header = None)
    rows, _ = np.where(np.array(u_best[1]) > 0.05)
    Labels = Meth_label.iloc[rows,:].reset_index(drop=True)
    Scores = pd.DataFrame(u_best[1][rows]).reset_index(drop=True)
    Meth_table = pd.concat([Labels, Scores],axis=1)
    Meth_table.columns = ['probes','scores']
    Meth_table = Meth_table.sort_values(by='scores', key=abs, ascending=False)

    Meth_total.append(Meth_table)

    miRNA_label = pd.read_csv('RealData/miRNA664_miRNA.txt', sep='\t',header = None)
    rows, _ = np.where(np.array(u_best[2]) > 0.05)
    Labels = miRNA_label.iloc[rows,:].reset_index(drop=True)
    Scores = pd.DataFrame(u_best[2][rows]).reset_index(drop=True)
    miRNA_table = pd.concat([Labels, Scores],axis=1)
    miRNA_table.columns = ['miRNA','scores']
    miRNA_table = miRNA_table.sort_values(by='scores', key=abs, ascending=False)

    miRNA_total.append(miRNA_table)

## Save
if not os.path.exists('Results'):
    os.makedirs('Results')
print(miRNA_total)
#Exp_table.to_csv('Results//Exp_score.csv', index=False)
#Meth_table.to_csv('Results//Meth_score.csv', index=False)
#miRNA_table.to_csv('Results//miRNA_score.csv', index=False)

[             miRNA    scores
13     hsa-mir-195  0.177013
19    hsa-mir-2114  0.153660
24    hsa-mir-3136  0.151416
33    hsa-mir-3676  0.134831
7      hsa-mir-154  0.131467
5     hsa-mir-147b  0.128066
17      hsa-mir-21  0.117542
49     hsa-mir-605  0.107413
54     hsa-mir-9-1  0.104081
53    hsa-mir-891a  0.104002
20     hsa-mir-223  0.103093
57     hsa-mir-940  0.100287
11     hsa-mir-188  0.098643
4     hsa-mir-1468  0.098063
51     hsa-mir-760  0.094904
3      hsa-mir-141  0.092707
34    hsa-mir-3678  0.087355
31    hsa-mir-3647  0.086158
6      hsa-mir-149  0.085493
12     hsa-mir-18a  0.084260
39     hsa-mir-451  0.082430
35     hsa-mir-377  0.078855
29     hsa-mir-342  0.075546
26  hsa-mir-3199-2  0.075359
27      hsa-mir-32  0.075145
46     hsa-mir-576  0.075052
2     hsa-mir-1306  0.074425
14  hsa-mir-196a-1  0.074085
1     hsa-mir-1266  0.073121
56     hsa-mir-939  0.071522
52     hsa-mir-874  0.069218
38     hsa-mir-432  0.068506
43   hsa-mir-514-3  0.067650
21     hsa-mi

In [56]:
from functools import reduce
result_Exp = reduce(lambda left, right: pd.merge(left, right, on='genes', how='outer'), Exp_total)
print(result_Exp.dropna())
result_Meth = reduce(lambda left, right: pd.merge(left, right, on='probes', how='outer'), Meth_total)
print(result_Meth)
result_miRNA = reduce(lambda left, right: pd.merge(left, right, on='miRNA', how='outer'), miRNA_total)
print(result_miRNA.dropna())


            genes  scores_x  scores_y  scores_x  scores_y    scores
1      ESPN|83715  0.078147  0.075832  0.083639  0.148606  0.085063
2       TPH1|7166  0.076751  0.053538  0.121870  0.085059  0.165028
4    BMPER|168667  0.074993  0.056737  0.145334  0.071424  0.140306
5    PTPRQ|374462  0.070078  0.072470  0.080310  0.098032  0.133508
10      STC1|6781  0.062597  0.057758  0.087356  0.066216  0.097404
14       ELN|2006  0.059007  0.054346  0.079500  0.075362  0.081541
16     AGR2|10551  0.056521  0.083574  0.073343  0.104565  0.079159
21  PGLYRP4|57115  0.052489  0.067026  0.062843  0.067121  0.080651
        probes  scores_x  scores_y  scores_x  scores_y    scores
0   cg27652350  0.149853       NaN  0.122351  0.121003  0.096488
1   cg13636404  0.079043       NaN       NaN       NaN  0.058667
2   cg23657409  0.079033       NaN  0.095839  0.064725  0.078835
3   cg26486663  0.076207       NaN  0.095327  0.062065  0.072601
4   cg24750391  0.074845       NaN  0.085702  0.060362  0.06207

C:\Users\Programer\AppData\Local\Temp\ipykernel_28352\1347659830.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'scores_x'} in the result is deprecated and will raise a MergeError in a future version.
  result_Exp = reduce(lambda left, right: pd.merge(left, right, on='genes', how='outer'), Exp_total)
C:\Users\Programer\AppData\Local\Temp\ipykernel_28352\1347659830.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'scores_x'} in the result is deprecated and will raise a MergeError in a future version.
  result_Meth = reduce(lambda left, right: pd.merge(left, right, on='probes', how='outer'), Meth_total)
C:\Users\Programer\AppData\Local\Temp\ipykernel_28352\1347659830.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'scores_x'} in the result is deprecated and will raise a MergeError in a future version.
  result_miRNA = reduce(lambda left, right: pd.merge(left, right, on='miRNA', how='outer'), miRNA_total)
